# **Trabajo Práctico Final 2025 - Procesamiento del Lenguaje Natural - TUIA**

**Alumna**: Valentina Balverdi

**Profesores:**
* Juan Pablo Manson
* Alan Geray
* Constantino Ferrucci


### Descarga de archivos desde Drive

In [ ]:
import os
import glob
import json
import numpy
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/TUIA/NLP/TP_FINAL/fuentes_de_informacion"

# Carga de archivos tabulares
path = BASE_PATH

devoluciones = pd.read_csv(os.path.join(path, "devoluciones.csv"))
inventario = pd.read_csv(os.path.join(path, "inventario_sucursales.csv"))
productos = pd.read_csv(os.path.join(path, "productos.csv"))
ventas = pd.read_csv(os.path.join(path, "ventas_historicas.csv"))
vendedores = pd.read_csv(os.path.join(path, "vendedores.csv"))
tickets = pd.read_csv(os.path.join(path, "tickets_soporte.csv"))

with open(os.path.join(path, "faqs.json"), "r", encoding="utf-8") as f:
    faqs = json.load(f)

Mounted at /content/drive


In [ ]:
tickets.head()

,id_ticket,fecha_apertura,id_venta,id_producto,nombre_producto,cliente_nombre,cliente_provincia,tipo_problema,descripcion,severidad,categoria,estado,vendedor_asignado,sucursal,fecha_resolucion,dias_resolucion,garantia_valida
0,TKT000001,2023-06-16,VTA000305,P0251,Pro Lavaseca,Lola Ruiz,Tucumán,Sobrecalentamiento,El producto se calienta excesivamente durante ...,Alta,Eléctrico,Resuelto,Adrián Reyes,Santiago del Estero,2023-06-17,1.0,Si
1,TKT000002,2023-03-24,VTA000570,P0086,Digital Olla de Cocción Lenta,Carlos Gómez,Tierra del Fuego,No cumple especificaciones,El rendimiento no coincide con las especificac...,Media,Rendimiento,Cerrado,Tomás Ramírez,San Luis,2023-03-28,4.0,Si
2,TKT000003,2024-07-14,VTA009780,P0132,Exprimidor,Facundo Aguilar,Buenos Aires,Sobrecalentamiento,El producto se calienta excesivamente durante ...,Alta,Eléctrico,Cerrado,Rodrigo Reyes,CABA,2024-07-15,1.0,Si
3,TKT000004,2024-03-30,VTA002945,P0191,Turbo Ventilador de Techo,Florencia Acosta,Corrientes,Defecto de fábrica,Defecto visible desde la compra,Alta,Calidad,Cerrado,Emilia Hernández,Chubut,2024-04-01,2.0,Si
4,TKT000005,2024-03-17,VTA005497,P0076,Deluxe Parrilla Eléctrica,Emanuel Rivera,Córdoba,Mal olor,El producto emite olores inusuales,Baja,General,Resuelto,Martina Rojas,Neuquén,2024-03-19,2.0,Si


In [ ]:
# Carga de textos
resenas_dir = os.path.join(path, "resenas_usuarios")
reviews_files = glob.glob(os.path.join(resenas_dir, "*.txt"))

reviews_docs = []

for fp in reviews_files:
    with open(fp, "r", encoding="utf-8") as f:
        text = f.read()

    # Separar encabezado de cuerpo (asumo una línea en blanco entre ambos)
    partes = text.split("\n\n", 1)
    header = partes[0]
    cuerpo = partes[1] if len(partes) > 1 else ""

    # Inicializar campos
    fecha = usuario = telefono = producto_nombre = producto_id = puntaje = provincia = None

    for line in header.splitlines():
        line = line.strip()
        if line.startswith("Fecha:"):
            fecha = line.replace("Fecha:", "").strip()
        elif line.startswith("Usuario:"):
            usuario = line.replace("Usuario:", "").strip()
        elif line.startswith("Teléfono:"):
            telefono = line.replace("Teléfono:", "").strip()
        elif line.startswith("Producto:"):
            # Ej: "Producto: Profesional Batidora de Mano (P0017)"
            m = re.match(r"Producto:\s*(.+)\s+\((P\d+)\)", line)
            if m:
                producto_nombre = m.group(1).strip()
                producto_id = m.group(2).strip()
        elif line.startswith("Puntaje:"):
            # Ej: "Puntaje: 5/5"
            puntaje = line.replace("Puntaje:", "").strip()
        elif line.startswith("Provincia:"):
            provincia = line.replace("Provincia:", "").strip()

    reviews_docs.append({
        "content": cuerpo.strip(),              # texto que se embebe
        "source": os.path.basename(fp),
        "type": "review",
        "fecha": fecha,
        "usuario": usuario,
        "telefono": telefono,
        "producto_id": producto_id,
        "producto_nombre": producto_nombre,
        "puntaje": puntaje,
        "provincia": provincia,
    })


In [ ]:

# Manuales .md
manuales_dir = os.path.join(path, "manuales_productos")
manuales_files = glob.glob(os.path.join(manuales_dir, "*.md"))

manuales_docs = []

for fp in manuales_files:
    with open(fp, "r", encoding="utf-8") as f:
        text = f.read()

    # --- Extraer información del encabezado ---
    # Título: "# Manual Técnico - Compacto Licuadora"
    titulo_match = re.search(r"# Manual Técnico - (.+)", text)
    producto_nombre = titulo_match.group(1).strip() if titulo_match else None

    # Modelo: "**Modelo:** P0004 | **Marca:** ChefMaster"
    modelo_match = re.search(r"\*\*Modelo:\*\*\s*([A-Z0-9]+)", text)
    producto_id = modelo_match.group(1).strip() if modelo_match else None

    marca_match = re.search(r"\*\*Marca:\*\*\s*([A-Za-z0-9]+)", text)
    marca = marca_match.group(1).strip() if marca_match else None

    # Nombre Comercial (en Especificaciones Técnicas)
    comercial_match = re.search(r"\*\*Nombre Comercial:\*\*\s*(.+)", text)
    nombre_comercial = comercial_match.group(1).strip() if comercial_match else producto_nombre

    # Categoría
    categoria_match = re.search(r"\*\*Categoría:\*\*\s*(.+)", text)
    categoria = categoria_match.group(1).strip() if categoria_match else None

    # Guardar documento con metadata
    manuales_docs.append({
        "content": text,
        "source": os.path.basename(fp),
        "type": "manual",
        "producto_id": producto_id,
        "producto_nombre": nombre_comercial,
        "marca": marca,
        "categoria_producto": categoria
    })

# Convertir FAQs a documentos
faq_docs = []

for item in faqs:
    pregunta = item["pregunta"]
    respuesta = item["respuesta"]
    producto = item["nombre_producto"]
    categoria = item["categoria"]

    faq_docs.append({
        "content": (
            f"PRODUCTO: {producto}\n"
            f"CATEGORÍA FAQ: {categoria}\n"
            f"PREGUNTA: {pregunta}\n"
            f"RESPUESTA: {respuesta}"
        ),
        "source": f"faq_{item['id_faq']}.json",
        "type": "faq",
        "id_faq": item["id_faq"],
        "id_producto": item["id_producto"],
        "producto_nombre": item["nombre_producto"],
        "categoria_faq": item["categoria"]
    })


# Tickets de soporte (texto largo)
ticket_docs = []

for _, row in tickets.iterrows():
    texto = ""

    # incluir tipo de problema si existe
    if isinstance(row["tipo_problema"], str):
        texto += f"Tipo de problema: {row['tipo_problema']}\n"

    # incluir descripción si existe
    if isinstance(row["descripcion"], str):
        texto += f"Descripción: {row['descripcion']}\n"

    # Si tenemos texto válido, lo guardamos
    if texto.strip():
        ticket_docs.append({
            "content": texto.strip(),
            "source": f"ticket_{row['id_ticket']}",
            "type": "ticket_soporte",
            "producto_id": row["id_producto"],
            "producto_nombre": row["nombre_producto"],
            "categoria": row["categoria"],
            "severidad": row["severidad"],
        })

# Union para la base vectorial
text_documents = reviews_docs + manuales_docs + faq_docs + ticket_docs


In [ ]:
print("=== FAQs ===")
print("Cantidad de FAQs:", len(faqs))
print("Ejemplo FAQ[0]:")
print(faqs[0])

print("\n=== ticket_docs ===")
print("Cantidad de ticket_docs:", len(ticket_docs))
print("Ejemplo ticket_docs[0]:")
print(ticket_docs[0])

print("\n=== DataFrames tabulares ===")
for nombre, df in {
    "devoluciones": devoluciones,
    "inventario": inventario,
    "productos": productos,
    "ventas": ventas,
    "vendedores": vendedores,
    "tickets_df": tickets,
}.items():
    print(f"\n[{nombre}] shape={df.shape}")
    print("Columnas:", list(df.columns))


=== FAQs ===
Cantidad de FAQs: 3000
Ejemplo FAQ[0]:
{'id_faq': 'FAQ00001', 'id_producto': 'P0001', 'nombre_producto': 'Licuadora', 'categoria': 'Especificaciones', 'pregunta': '¿Qué voltaje requiere?', 'respuesta': 'El Licuadora funciona con 12V. El consumo es de 650W. Recomendamos usar un estabilizador de tensión.', 'fecha_publicacion': '2025-01-05', 'vistas': 4067, 'util': 22}

=== ticket_docs ===
Cantidad de ticket_docs: 2000
Ejemplo ticket_docs[0]:
{'content': 'Tipo de problema: Sobrecalentamiento\nDescripción: El producto se calienta excesivamente durante el uso', 'source': 'ticket_TKT000001', 'type': 'ticket_soporte', 'producto_id': 'P0251', 'producto_nombre': 'Pro Lavaseca', 'categoria': 'Eléctrico', 'severidad': 'Alta'}

=== DataFrames tabulares ===

[devoluciones] shape=(800, 14)
Columnas: ['id_devolucion', 'id_venta', 'fecha_devolucion', 'id_producto', 'nombre_producto', 'cliente_nombre', 'motivo', 'descripcion_cliente', 'estado', 'monto_venta', 'monto_reembolso', 'metodo_ree

In [ ]:
def stats_longitudes(docs, nombre):
    longitudes = [len(d["content"]) for d in docs]
    if not longitudes:
        print(f"No hay documentos en {nombre}")
        return None

    serie = pd.Series(longitudes)
    print(f"=== {nombre} ===")
    print("Cantidad de documentos:", len(longitudes))
    print("Mínimo:", int(serie.min()))
    print("Máximo:", int(serie.max()))
    print("Promedio:", int(serie.mean()))
    print("Mediana:", int(serie.median()))
    print("Percentil 75:", int(serie.quantile(0.75)))
    print("Percentil 90:", int(serie.quantile(0.90)))
    print()
    return serie

len_reviews  = stats_longitudes(reviews_docs,  "Reseñas")
len_manuales = stats_longitudes(manuales_docs, "Manuales")
len_faqs     = stats_longitudes(faq_docs,      "FAQs")
len_tickets  = stats_longitudes(ticket_docs,   "Tickets soporte")


=== Reseñas ===
Cantidad de documentos: 5015
Mínimo: 132
Máximo: 282
Promedio: 200
Mediana: 203
Percentil 75: 223
Percentil 90: 237

=== Manuales ===
Cantidad de documentos: 50
Mínimo: 6450
Máximo: 8675
Promedio: 6623
Mediana: 6516
Percentil 75: 6539
Percentil 90: 6968

=== FAQs ===
Cantidad de documentos: 3000
Mínimo: 190
Máximo: 365
Promedio: 302
Mediana: 308
Percentil 75: 321
Percentil 90: 332

=== Tickets soporte ===
Cantidad de documentos: 2000
Mínimo: 74
Máximo: 109
Promedio: 91
Mediana: 92
Percentil 75: 102
Percentil 90: 109



# Ejercicio 1: RAG

## **Base de datos Vectorial**
La base de datos vectorial se utiliza para indexar todas las fuentes de información no estructurada o semiestructurada del proyecto, incluyendo:

- Manuales de producto (`.md`)
- Reseñas de usuarios (`.txt`)
- Preguntas frecuentes (`faqs.json`)
- Descripciones de tickets de soporte (`tickets_soporte.csv`)

Este tipo de contenido es ideal para un esquema (RAG), ya que los usuarios realizan consultas en lenguaje natural y se requiere una búsqueda semántica, no solo coincidencia literal de palabras clave.

### **Fragmentación del texto (Text Splitter)**

Para preparar los documentos antes de generar embeddings, se utilizó el **RecursiveCharacterTextSplitter** de LangChain. Este splitter fue elegido porque:

- Es robusto para textos largos como manuales.
- Intenta mantener unidades semánticas coherentes (párrafos, oraciones) antes de cortar por caracteres.
- Permite controlar el tamaño del fragmento (*chunk_size*) y el solapamiento (*chunk_overlap*).
- Evita cortes innecesarios en textos cortos como reseñas, FAQs y tickets.

A partir del análisis de longitudes reales de los documentos:

| Fuente      | P75 longitud |
|-------------|--------------|
| Reseñas     | 368 chars    |
| FAQs        | 317 chars    |
| Tickets     | 102 chars    |
| Manuales    | 6539 chars   |

Se seleccionó:

- **chunk_size = 512** → suficientemente grande para mantener intactas reseñas, FAQs y tickets,  
  pero suficientemente pequeño para dividir manuales extensos de forma manejable.
- **chunk_overlap = 64** (≈ 12%) → asegura continuidad semántica entre fragmentos contiguos.

In [ ]:
!pip install langchain-text-splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=512,
    chunk_overlap=64,
)

In [ ]:
chunks_final = []

# ---- Reviews ----
for i, review_doc in enumerate(reviews_docs):
    chunks = splitter.split_text(review_doc["content"])
    for j, chunk in enumerate(chunks):
        chunks_final.append({
            "id": f"review_{i}_chunk_{j}",
            "content": chunk,
            "metadata": {
                "type": "review",
                "fuente": review_doc["source"],
                "chunk_index": j,
                "producto_id": review_doc.get("producto_id"),
                "producto_nombre": review_doc.get("producto_nombre"),
                "puntaje": review_doc.get("puntaje"),
                "provincia": review_doc.get("provincia"),
            }
        })

# ---- Manuales ----
for i, manual_doc in enumerate(manuales_docs):
    chunks = splitter.split_text(manual_doc["content"])
    for j, chunk in enumerate(chunks):
        chunks_final.append({
            "id": f"manual_{i}_chunk_{j}",
            "content": chunk,
            "metadata": {
                "type": "manual",
                "fuente": manual_doc["source"],
                "chunk_index": j,
                "producto_id": manual_doc.get("producto_id"),
                "producto_nombre": manual_doc.get("producto_nombre"),
                "marca": manual_doc.get("marca"),
                "categoria_producto": manual_doc.get("categoria_producto"),
            }
        })

# ---- FAQs ----
for i, faq_doc in enumerate(faq_docs):
    chunks = splitter.split_text(faq_doc["content"])
    for j, chunk in enumerate(chunks):
        chunks_final.append({
            "id": f"faq_{i}_chunk_{j}",
            "content": chunk,
            "metadata": {
                "type": "faq",
                "fuente": faq_doc["source"],
                "chunk_index": j,
                "id_faq": faq_doc.get("id_faq"),
                "id_producto": faq_doc.get("id_producto"),
                "producto_nombre": faq_doc.get("producto_nombre"),
                "categoria_faq": faq_doc.get("categoria_faq"),
            }
        })

# ---- Tickets ----
for i, ticket_doc in enumerate(ticket_docs):
    chunks = splitter.split_text(ticket_doc["content"])
    for j, chunk in enumerate(chunks):
        chunks_final.append({
            "id": f"ticket_{i}_chunk_{j}",
            "content": chunk,
            "metadata": {
                "type": "ticket",
                "fuente": ticket_doc["source"],
                "chunk_index": j,
                "producto_id": ticket_doc.get("producto_id"),
                "producto_nombre": ticket_doc.get("producto_nombre"),
                "categoria_ticket": ticket_doc.get("categoria"),
                "severidad": ticket_doc.get("severidad"),
            }
        })

print("Cantidad total de chunks:", len(chunks_final))
chunks_final[0]


Cantidad total de chunks: 10839


{'id': 'review_0_chunk_0',
 'content': 'Buenas! Llegó rápido y funciona de maravilla con Aire Portátil Pro. Lo compré hace una semana y es muy fácil de usar, además de moderno. Ya se lo recomendé a varios amigos. Saludos!',
 'metadata': {'type': 'review',
  'fuente': 'resena_R03514.txt',
  'chunk_index': 0,
  'producto_id': 'P0156',
  'producto_nombre': 'Aire Portátil Pro',
  'puntaje': '5/5',
  'provincia': 'Neuquén'}}

### **Modelo de Embeddings**

Para transformar cada fragmento en un vector numérico se seleccionó:

Modelo elegido: **sentence-transformers/paraphrase-multilingual-mpnet-base-v2**

Justificación:

- Es un modelo multilingüe optimizado para español.
- Destacado en tareas de similitud semántica y sistemas RAG.
- Produce embeddings densos de 768 dimensiones con alta calidad semántica.
- Eficiente
- Permite comparar consultas y documentos en el mismo espacio vectorial.

In [ ]:
!pip install sentence-transformers chromadb

In [ ]:
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

documents = [c["content"] for c in chunks_final]
metadatas = [c["metadata"] for c in chunks_final]
ids       = [c["id"]       for c in chunks_final]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = embedding_model.encode(
    documents,
    batch_size=64,
    show_progress_bar=True
)

Batches:   0%|          | 0/170 [00:00<?, ?it/s]

### **Almacenamiento en ChromaDB**

Se utilizó ChromaDB con almacenamiento persistente, configurado con distancia coseno

In [ ]:
!pip install chromadb


In [ ]:
import chromadb
from chromadb.config import Settings

chroma_client = chromadb.PersistentClient(path="chroma_vector_db")

# Creamos una nueva colección
collection = chroma_client.get_or_create_collection(
    name="electrodomesticos_vector_store",
    metadata={"hnsw:space": "cosine"}
)

In [ ]:
# Debido a una limitación interna del cliente Rust, la colección se carga en dos batch:
n = len(documents)
MAX_BATCH = 5461

print(f"Total de documentos a insertar: {n}")

print("Cargando primer batch...")
collection.add(
    documents=documents[:MAX_BATCH],
    metadatas=metadatas[:MAX_BATCH],
    ids=ids[:MAX_BATCH],
    embeddings=embeddings[:MAX_BATCH]
)

if n > MAX_BATCH:
    print("Cargando segundo batch...")
    collection.add(
        documents=documents[MAX_BATCH:],
        metadatas=metadatas[MAX_BATCH:],
        ids=ids[MAX_BATCH:],
        embeddings=embeddings[MAX_BATCH:]
    )

print("Carga finalizada.")
print("Documentos totales en Chroma:", collection.count())


Total de documentos a insertar: 10839
Cargando primer batch...
Cargando segundo batch...
Carga finalizada.
Documentos totales en Chroma: 10839


**Función de Búsqueda Vectorial**

Esta función es el método que utiliza el sistema para recuperar fragmentos relevantes.
Incluye:
- Normalización de filtros ($and automático)
- Manejo de errores
- Ordenamiento por relevancia
- Soporte opcional para embeddings de los documentos (útil para reranking)

In [ ]:
def normalizar_filtros(filtros):
    """
    Normaliza filtros para ChromaDB.
    Si el usuario pasa un dict simple como {"type": "manual", "producto":"X"},
    lo convierte automáticamente a {"$and": [{"type":"manual"}, {"producto":"X"}]}
    """
    if filtros is None:
        return None

    # Si ya viene con operador ($and / $or / $not), lo dejamos como está
    if any(key.startswith("$") for key in filtros.keys()):
        return filtros

    # Convertimos dict simple → $and
    condiciones = [{k: v} for k, v in filtros.items()]
    return {"$and": condiciones}


def buscar_vectorial(consulta, k=5, filtros=None, devolver_embeddings=False):
    """
    Realiza búsqueda vectorial en ChromaDB.

    Parámetros:
        consulta (str): texto de la consulta del usuario
        k (int): cantidad de resultados a recuperar
        filtros (dict): condiciones para filtrar metadata
        devolver_embeddings (bool): si se desea incluir embeddings del resultado
                                     (útil para rerankers tipo CrossEncoder)

    Devuelve:
        lista de dicts: cada uno con id, contenido, metadata y distancia
    """
    # 1. Embedding de la consulta
    query_emb = embedding_model.encode([consulta])[0]

    # 2. Normalizar filtros a formato aceptado por Chroma
    where = normalizar_filtros(filtros)

    # 3. Ejecutar consulta en Chroma
    if where:
        result = collection.query(
            query_embeddings=[query_emb],
            n_results=k,
            where=where
        )
    else:
        result = collection.query(
            query_embeddings=[query_emb],
            n_results=k
        )

    # 4. Si no encuentra nada
    if len(result["ids"][0]) == 0:
        return []

    # 5. Construir resultados limpios
    resultados = []
    for i in range(len(result["ids"][0])):
        entrada = {
            "id": result["ids"][0][i],
            "content": result["documents"][0][i],
            "metadata": result["metadatas"][0][i],
            "distance": result["distances"][0][i]
        }

        if devolver_embeddings:
            entrada["embedding_doc"] = result["embeddings"][0][i]

        resultados.append(entrada)

    # 6. Ordenar por distancia (menor = más relevante)
    resultados.sort(key=lambda x: x["distance"])

    return resultados


In [ ]:
prueba = buscar_vectorial(
    "¿Cómo usar la licuadora para hacer un smoothie?",
    k=3,
    filtros={"type": "manual", "producto_nombre": "Compacto Licuadora"}
)

for r in prueba:
    print(r["metadata"])
    print(r["content"][:200], "...\n---")


{'marca': 'ChefMaster', 'type': 'manual', 'producto_nombre': 'Compacto Licuadora', 'fuente': 'manual_P0004_Compacto_Licuadora.md', 'chunk_index': 5, 'producto_id': 'P0004', 'categoria_producto': 'Cocina - Preparación'}
## Procedimientos de Uso

### PROCEDIMIENTO 1: Preparar Smoothie de Frutas

**Dificultad:** Fácil | **Tiempo:** 3-5 minutos

**Pasos:** ...
---
{'chunk_index': 6, 'producto_id': 'P0004', 'categoria_producto': 'Cocina - Preparación', 'marca': 'ChefMaster', 'producto_nombre': 'Compacto Licuadora', 'fuente': 'manual_P0004_Compacto_Licuadora.md', 'type': 'manual'}
1. Lavar y cortar las frutas en trozos medianos (2-3 cm)
2. Colocar los ingredientes líquidos primero (leche, yogurt, jugo)
3. Agregar las frutas y hielo en la jarra
4. Cerrar la tapa asegurándose del ...
---
{'chunk_index': 7, 'fuente': 'manual_P0004_Compacto_Licuadora.md', 'categoria_producto': 'Cocina - Preparación', 'type': 'manual', 'producto_id': 'P0004', 'producto_nombre': 'Compacto Licuadora', 'marca': 'Che

## **Base Tabular** (datos estadísticos)
Para resolver consultas donde el usuario necesita información estructurada.

Las tablas provienen de los archivos CSV del dataset:
- productos.csv
- ventas_historicas.csv
- inventario_sucursales.csv
- devoluciones.csv
- vendedores.csv
- tickets_soporte.csv

In [ ]:
# Agrupamos las tablas que vamos a usar como fuente tabular
tablas_tabulares = {
    "productos": productos,
    "inventario": inventario,
    "ventas": ventas,
    "devoluciones": devoluciones,
    "vendedores": vendedores,
    "tickets": tickets,
}

{nombre: df.shape for nombre, df in tablas_tabulares.items()}


{'productos': (300, 14),
 'inventario': (4100, 14),
 'ventas': (10000, 15),
 'devoluciones': (800, 14),
 'vendedores': (100, 10),
 'tickets': (2000, 17)}

In [ ]:
def generar_estructura_para_llm(nombre_df: str, df: pd.DataFrame, max_uniques: int = 20):
    """
    Genera una estructura compacta con información relevante de la tabla,
    pensada para ser convertida a string y enviada a un LLM.

    Incluye:
    - nombre de la tabla
    - columnas
    - tipos de dato
    - cantidad de filas / columnas
    - cantidad de nulos por columna
    - resumen por columna:
        - numéricas: min / max
        - categóricas: valores únicos (solo si son pocos)
        - texto: solo se marca como texto
    - estadísticas numéricas generales (describe), por si se quiere usar aparte
    """

    estructura = {
        "nombre_df": nombre_df,
        "cantidad_filas": int(len(df)),
        "cantidad_columnas": int(df.shape[1]),
        "columnas": df.columns.tolist(),
        "tipos_de_dato": df.dtypes.astype(str).to_dict(),
        "valores_nulos": df.isnull().sum().astype(int).to_dict(),
        "resumen_columnas": {},
        "estadisticas_numericas": {}
    }

    # Resumen por columna
    for col in df.columns:
        serie = df[col]
        col_info = {}

        if pd.api.types.is_numeric_dtype(serie):
            col_info["tipo_logico"] = "numerico"
            col_info["min"] = float(serie.min()) if not serie.isna().all() else None
            col_info["max"] = float(serie.max()) if not serie.isna().all() else None

        elif pd.api.types.is_object_dtype(serie) or pd.api.types.is_categorical_dtype(serie):
            # Columna categórica / texto
            n_unique = serie.nunique(dropna=True)
            col_info["tipo_logico"] = "categorico" if n_unique <= max_uniques else "texto_largo"
            col_info["num_valores_unicos"] = int(n_unique)

            if n_unique <= max_uniques:
                col_info["valores_unicos"] = serie.dropna().unique().tolist()

        else:
            col_info["tipo_logico"] = "otro"

        estructura["resumen_columnas"][col] = col_info

    # Estadísticas numéricas (describe)
    if not df.select_dtypes(include="number").empty:
        estructura["estadisticas_numericas"] = (
            df.select_dtypes(include="number").describe().to_dict()
        )

    return estructura


In [ ]:
def estructura_a_string(estructura: dict) -> str:
    """
    Convierte la estructura generada por generar_estructura_para_llm
    a un string legible para usar en el prompt del LLM.
    """
    lineas = []
    lineas.append(f"Nombre de la tabla: {estructura['nombre_df']}")
    lineas.append(f"Filas: {estructura['cantidad_filas']}, Columnas: {estructura['cantidad_columnas']}")
    lineas.append("Columnas:")

    for col, info in estructura["resumen_columnas"].items():
        tipo = info.get("tipo_logico", "desconocido")

        if tipo == "numerico":
            lineas.append(
                f"- {col}: numérico, rango aproximado [{info.get('min')}, {info.get('max')}]"
            )
        elif tipo == "categorico":
            vals = ", ".join(map(str, info.get("valores_unicos", [])))
            lineas.append(
                f"- {col}: categórico, valores posibles: {vals}"
            )
        elif tipo == "texto_largo":
            lineas.append(
                f"- {col}: texto libre / muchas categorías (≈{info.get('num_valores_unicos')} valores distintos)"
            )
        else:
            lineas.append(f"- {col}: tipo {tipo}")

    return "\n".join(lineas)


In [ ]:
import json

# 1) Estructuras "ricas" (diccionarios)
esquemas_tabulares = {
    nombre: generar_estructura_para_llm(nombre, df)
    for nombre, df in tablas_tabulares.items()
}

# 2) Versión legible como texto para meter en el prompt del LLM
descripciones_tablas = {
    nombre: estructura_a_string(estructura)
    for nombre, estructura in esquemas_tabulares.items()
}



Nombre de la tabla: productos
Filas: 300, Columnas: 14
Columnas:
- id_producto: texto libre / muchas categorías (≈300 valores distintos)
- nombre: texto libre / muchas categorías (≈247 valores distintos)
- categoria: categórico, valores posibles: Cocina, Climatización, Lavado, Audio y Video
- subcategoria: categórico, valores posibles: Preparación, Cocción, Refrigeración, Pequeños Electrodomésticos, Aires Acondicionados, Calefacción, Ventilación, Purificación, Lavado de Ropa, Secado, Lavado de Vajilla, Planchado, Televisores
- marca: categórico, valores posibles: TechHome, ChefMaster, HomeChef, KitchenPro, CookElite, PureAir, EcoClima, ClimaTech, ThermoControl, AirFlow, WashPro, SparkleHome, CleanMaster, LaundryTech, FreshWash, VisionPro, ScreenPro
- precio_usd: numérico, rango aproximado [28.22, 2992.33]
- stock: numérico, rango aproximado [1.0, 200.0]
- color: categórico, valores posibles: Blanco, Rosa, Negro, Azul, Dorado, Gris, Plateado, Verde, Rojo, Amarillo
- potencia_w: numérico

In [ ]:
def armar_prompt_tabular(query_usuario: str) -> str:
    """
    Construye el prompt que se le va a enviar al LLM para que
    genere código de pandas que responda a la consulta del usuario.
    """

    partes = []

    partes.append(
        "Sos un asistente experto en análisis de datos con pandas.\n"
        "Trabajás con datos de una empresa de electrodomésticos.\n\n"
        "Tu tarea es: dado una pregunta del usuario, responder EXCLUSIVAMENTE con código Python\n"
        "que use DataFrames de pandas ya cargados para obtener la respuesta.\n\n"
        "IMPORTANTE:\n"
        "- NO crees DataFrames nuevos desde cero con datos escritos a mano.\n"
        "- NO uses read_csv ni lecturas de archivos.\n"
        "- NO imprimas nada, NO uses print.\n"
        "- NO expliques el código, solo devolvé la expresión de pandas.\n"
        "- La expresión debe devolver directamente el resultado (DataFrame o valor escalar).\n"
        "- Los DataFrames disponibles son:\n"
        "  * productos\n"
        "  * inventario\n"
        "  * ventas\n"
        "  * devoluciones\n"
        "  * vendedores\n"
        "  * tickets\n"
    )

    partes.append("\nA continuación te doy un resumen de las tablas disponibles:\n")

    for nombre, desc in descripciones_tablas.items():
        partes.append(f"\n### Tabla: {nombre}\n{desc}\n")

    partes.append("\nPregunta del usuario:\n")
    partes.append(query_usuario)
    partes.append(
        "\n\nAhora respondé ÚNICAMENTE con código Python válido que utilice pandas "
        "y estos DataFrames para obtener la respuesta. No agregues explicaciones ni comentarios."
    )

    return "\n".join(partes)


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama pull llama3.1


Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [ ]:
def llamar_llm_tabular(prompt: str) -> str:
    """
    Esta función es el ÚNICO lugar donde se llama al modelo de lenguaje.
    Acá podés implementar:
      - Gemini (google.genai)
      - OpenAI (openai / openai-python)
      - Ollama (ollama.chat)

    Debe devolver SOLO el código Python como string.
    Sin ```python, sin explicaciones, sin texto extra.
    """
    # EJEMPLO (pseudo-código, no se ejecuta):
    #
    # from google import genai
    # client = genai.Client(api_key="TU_API_KEY")
    # resp = client.models.generate_content(
    #     model="gemini-2.0-flash",
    #     contents=prompt
    # )
    # codigo = resp.text
    # return codigo
    #
    # Por ahora lo dejamo sin implementar:
    raise NotImplementedError("Implementar esta función con el LLM elegido (Gemini, OpenAI u Ollama).")


In [ ]:
def generar_codigo_pandas_desde_llm(query_usuario: str) -> str:
    """
    Toma la consulta en lenguaje natural, arma el prompt completo
    y usa el LLM para obtener SOLO código de pandas como respuesta.
    """
    prompt = armar_prompt_tabular(query_usuario)
    codigo = llamar_llm_tabular(prompt)

    # Por si el modelo devuelve ```python ... ```
    codigo_limpio = (
        codigo.replace("```python", "")
              .replace("```", "")
              .strip()
    )
    return codigo_limpio


In [ ]:
# Entorno seguro con las tablas reales
entorno_tablas = {
    "productos": productos,
    "inventario": inventario,
    "ventas": ventas,
    "devoluciones": devoluciones,
    "vendedores": vendedores,
    "tickets": tickets,
    "pd": pd,  # por si el modelo usa pd.X
}

def consulta_tabular(query_usuario: str):
    """
    - Llama al LLM para obtener código pandas.
    - Ejecuta ese código en un entorno controlado con las tablas.
    - Devuelve (resultado, codigo_generado).
    """
    codigo = generar_codigo_pandas_desde_llm(query_usuario)
    print("=== Código generado por el LLM ===")
    print(codigo)
    print("=================================")

    try:
        resultado = eval(codigo, {}, entorno_tablas)
    except Exception as e:
        print("⚠️ Error al ejecutar el código generado:")
        print(e)
        resultado = None

    return resultado, codigo


---

## Base de datos de grafos



In [ ]:
import pandas as pd

# ================================
# 1) DataFrames de NODOS
# ================================

# Productos
nodos_productos = productos[[
    "id_producto", "nombre", "categoria", "subcategoria",
    "marca", "precio_usd", "stock", "color", "potencia_w",
    "capacidad", "voltaje", "peso_kg", "garantia_meses"
]].copy()
nodos_productos.rename(columns={"id_producto": "id"}, inplace=True)
nodos_productos["label"] = "Producto"

# Ventas
nodos_ventas = ventas[[
    "id_venta", "fecha", "hora", "sucursal",
    "cantidad", "precio_unitario", "descuento_pct", "total",
    "metodo_pago"
]].copy()
nodos_ventas.rename(columns={"id_venta": "id"}, inplace=True)
nodos_ventas["label"] = "Venta"

# Clientes (a partir de ventas / devoluciones / tickets)
clientes_ventas = ventas["cliente_nombre"].dropna().unique()
clientes_devol = devoluciones["cliente_nombre"].dropna().unique()
clientes_tickets = tickets["cliente_nombre"].dropna().unique()

clientes_unicos = pd.Series(list(set(clientes_ventas) | set(clientes_devol) | set(clientes_tickets)))
nodos_clientes = pd.DataFrame({
    "id": clientes_unicos,
    "label": "Cliente"
})

# Tickets
nodos_tickets = tickets[[
    "id_ticket", "fecha_apertura", "tipo_problema",
    "descripcion", "severidad", "categoria", "estado",
    "sucursal", "fecha_resolucion", "dias_resolucion", "garantia_valida"
]].copy()
nodos_tickets.rename(columns={"id_ticket": "id"}, inplace=True)
nodos_tickets["label"] = "Ticket"

# Devoluciones
nodos_devoluciones = devoluciones[[
    "id_devolucion", "fecha_devolucion", "motivo",
    "descripcion_cliente", "estado", "monto_venta",
    "monto_reembolso", "metodo_reembolso", "fecha_reembolso"
]].copy()
nodos_devoluciones.rename(columns={"id_devolucion": "id"}, inplace=True)
nodos_devoluciones["label"] = "Devolucion"

# FAQs (desde faqs.json)
nodos_faqs = pd.DataFrame(faqs)
nodos_faqs = nodos_faqs[[
    "id_faq", "id_producto", "nombre_producto",
    "categoria", "pregunta", "respuesta",
    "fecha_publicacion", "vistas", "util"
]].copy()
nodos_faqs.rename(columns={"id_faq": "id"}, inplace=True)
nodos_faqs["label"] = "FAQ"

# Reseñas de usuarios (desde reviews_docs)
resenas_rows = []
for i, d in enumerate(reviews_docs):
    resenas_rows.append({
        "id": f"RESENA_{i}",
        "texto": d["content"],
        "fecha": d.get("fecha"),
        "usuario": d.get("usuario"),
        "telefono": d.get("telefono"),
        "producto_id": d.get("producto_id"),
        "producto_nombre": d.get("producto_nombre"),
        "puntaje": d.get("puntaje"),
        "provincia": d.get("provincia"),
        "label": "Resena"
    })

nodos_resenas = pd.DataFrame(resenas_rows)

# Manuales (desde manuales_docs)
manuales_rows = []
for i, d in enumerate(manuales_docs):
    producto_id = d.get("producto_id")
    manual_id = f"MAN_{producto_id}" if producto_id else f"MAN_{i}"
    manuales_rows.append({
        "id": manual_id,
        "producto_id": producto_id,
        "producto_nombre": d.get("producto_nombre"),
        "marca": d.get("marca"),
        "categoria_producto": d.get("categoria_producto"),
        "archivo": d.get("source"),
        "label": "Manual"
    })

nodos_manuales = pd.DataFrame(manuales_rows)

# ================================
# 2) DataFrames de RELACIONES
# ================================

# Producto - Venta
rel_prod_venta = ventas[["id_producto", "id_venta"]].copy()
rel_prod_venta.rename(columns={"id_producto": "from_id", "id_venta": "to_id"}, inplace=True)
rel_prod_venta["from_label"] = "Producto"
rel_prod_venta["to_label"] = "Venta"
rel_prod_venta["rel_type"] = "TIENE_VENTA"

# Venta - Cliente
rel_venta_cliente = ventas[["id_venta", "cliente_nombre"]].copy()
rel_venta_cliente.rename(columns={"id_venta": "from_id", "cliente_nombre": "to_id"}, inplace=True)
rel_venta_cliente["from_label"] = "Venta"
rel_venta_cliente["to_label"] = "Cliente"
rel_venta_cliente["rel_type"] = "A_CLIENTE"

# Producto - Ticket
rel_prod_ticket = tickets[["id_producto", "id_ticket"]].copy()
rel_prod_ticket.rename(columns={"id_producto": "from_id", "id_ticket": "to_id"}, inplace=True)
rel_prod_ticket["from_label"] = "Producto"
rel_prod_ticket["to_label"] = "Ticket"
rel_prod_ticket["rel_type"] = "TIENE_TICKET"

# Ticket - Venta
rel_ticket_venta = tickets[["id_ticket", "id_venta"]].copy()
rel_ticket_venta.rename(columns={"id_ticket": "from_id", "id_venta": "to_id"}, inplace=True)
rel_ticket_venta["from_label"] = "Ticket"
rel_ticket_venta["to_label"] = "Venta"
rel_ticket_venta["rel_type"] = "SOBRE_VENTA"

# Ticket - Cliente
rel_ticket_cliente = tickets[["id_ticket", "cliente_nombre"]].copy()
rel_ticket_cliente.rename(columns={"id_ticket": "from_id", "cliente_nombre": "to_id"}, inplace=True)
rel_ticket_cliente["from_label"] = "Ticket"
rel_ticket_cliente["to_label"] = "Cliente"
rel_ticket_cliente["rel_type"] = "DE_CLIENTE"

# Producto - Devolucion
rel_prod_devolucion = devoluciones[["id_producto", "id_devolucion"]].copy()
rel_prod_devolucion.rename(columns={"id_producto": "from_id", "id_devolucion": "to_id"}, inplace=True)
rel_prod_devolucion["from_label"] = "Producto"
rel_prod_devolucion["to_label"] = "Devolucion"
rel_prod_devolucion["rel_type"] = "TIENE_DEVOLUCION"

# Devolucion - Venta
rel_devolucion_venta = devoluciones[["id_devolucion", "id_venta"]].copy()
rel_devolucion_venta.rename(columns={"id_devolucion": "from_id", "id_venta": "to_id"}, inplace=True)
rel_devolucion_venta["from_label"] = "Devolucion"
rel_devolucion_venta["to_label"] = "Venta"
rel_devolucion_venta["rel_type"] = "SOBRE_VENTA"

# Devolucion - Cliente
rel_devolucion_cliente = devoluciones[["id_devolucion", "cliente_nombre"]].copy()
rel_devolucion_cliente.rename(columns={"id_devolucion": "from_id", "cliente_nombre": "to_id"}, inplace=True)
rel_devolucion_cliente["from_label"] = "Devolucion"
rel_devolucion_cliente["to_label"] = "Cliente"
rel_devolucion_cliente["rel_type"] = "DE_CLIENTE"

# Producto - FAQ
rel_prod_faq = nodos_faqs[["id_producto", "id"]].copy()
rel_prod_faq.rename(columns={"id_producto": "from_id", "id": "to_id"}, inplace=True)
rel_prod_faq["from_label"] = "Producto"
rel_prod_faq["to_label"] = "FAQ"
rel_prod_faq["rel_type"] = "TIENE_FAQ"

# Producto - Reseña
rel_prod_resena_rows = []
for i, d in enumerate(reviews_docs):
    if d.get("producto_id"):
        rel_prod_resena_rows.append({
            "from_id": d["producto_id"],
            "to_id": f"RESENA_{i}",
            "from_label": "Producto",
            "to_label": "Resena",
            "rel_type": "TIENE_RESENA",
        })

rel_prod_resena = pd.DataFrame(rel_prod_resena_rows)

# Producto - Manual
rel_prod_manual_rows = []
for i, d in enumerate(manuales_docs):
    prod_id = d.get("producto_id")
    if prod_id:
        manual_id = f"MAN_{prod_id}"
        rel_prod_manual_rows.append({
            "from_id": prod_id,
            "to_id": manual_id,
            "from_label": "Producto",
            "to_label": "Manual",
            "rel_type": "TIENE_MANUAL",
        })

rel_prod_manual = pd.DataFrame(rel_prod_manual_rows)

# Unimos todo en un único DataFrame de relaciones (para el informe y/o carga)
relaciones_grafo = pd.concat([
    rel_prod_venta,
    rel_venta_cliente,
    rel_prod_ticket,
    rel_ticket_venta,
    rel_ticket_cliente,
    rel_prod_devolucion,
    rel_devolucion_venta,
    rel_devolucion_cliente,
    rel_prod_faq,
    rel_prod_resena,
    rel_prod_manual,
], ignore_index=True)

print("Nodos productos:", nodos_productos.shape)
print("Nodos ventas:", nodos_ventas.shape)
print("Nodos clientes:", nodos_clientes.shape)
print("Nodos tickets:", nodos_tickets.shape)
print("Nodos devoluciones:", nodos_devoluciones.shape)
print("Nodos faqs:", nodos_faqs.shape)
print("Nodos reseñas:", nodos_resenas.shape)
print("Nodos manuales:", nodos_manuales.shape)
print("Total relaciones grafo:", relaciones_grafo.shape)


Nodos productos: (300, 14)
Nodos ventas: (10000, 10)
Nodos clientes: (2083, 2)
Nodos tickets: (2000, 12)
Nodos devoluciones: (800, 10)
Nodos faqs: (3000, 10)
Nodos reseñas: (5015, 10)
Nodos manuales: (50, 7)
Total relaciones grafo: (36465, 5)


### 🕸️ Creación del grafo con Neo4j

In [ ]:
!pip install py2neo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
from py2neo import Graph, Node, Relationship

NEO4J_URI = "neo4j+s://1830c5fb.databases.neo4j.io"  # algo tipo neo4j+s://xxxxx.databases.neo4j.io
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "foixXO3ES93u0U3tlAR0qVMDng1DQEHzMo8USaAIc_0"

In [ ]:
from py2neo import Node, Relationship

def limpiar_grafo():
    """Borra todos los nodos y relaciones de la base de Neo4j."""
    if graph is None:
        print("⚠️ No hay conexión a Neo4j")
        return
    graph.run("MATCH (n) DETACH DELETE n")
    print("🧹 Grafo limpiado")


def construir_grafo():
    """
    Construye un grafo sencillo con:
    Nodos: Producto, Ticket, Devolucion, FAQ
    Relaciones:
      (Producto)-[:TIENE_TICKET]->(Ticket)
      (Producto)-[:TIENE_DEVOLUCION]->(Devolucion)
      (Producto)-[:TIENE_FAQ]->(FAQ)
    Usa TODAS las filas de las tablas.
    """
    if graph is None:
        print("⚠️ No hay conexión a Neo4j")
        return

    limpiar_grafo()

    # ========= NODOS =========

    # Productos
    for _, row in productos.iterrows():
        nodo = Node(
            "Producto",
            id=row["id_producto"],
            nombre=row["nombre"],
            categoria=row["categoria"],
            subcategoria=row["subcategoria"],
            marca=row["marca"],
        )
        graph.merge(nodo, "Producto", "id")

    # Tickets
    for _, row in tickets.iterrows():
        nodo = Node(
            "Ticket",
            id=row["id_ticket"],
            tipo_problema=row["tipo_problema"],
            severidad=row["severidad"],
            categoria=row["categoria"],
            estado=row["estado"],
        )
        graph.merge(nodo, "Ticket", "id")

    # Devoluciones
    for _, row in devoluciones.iterrows():
        nodo = Node(
            "Devolucion",
            id=row["id_devolucion"],
            motivo=row["motivo"],
            estado=row["estado"],
            monto_reembolso=float(row["monto_reembolso"]),
        )
        graph.merge(nodo, "Devolucion", "id")

    # FAQs (viene de la lista 'faqs' que ya cargaste del JSON)
    for item in faqs:
        nodo = Node(
            "FAQ",
            id=item["id_faq"],
            categoria=item["categoria"],
            pregunta=item["pregunta"],
            respuesta=item["respuesta"],
        )
        graph.merge(nodo, "FAQ", "id")

    print("✅ Nodos creados (Producto, Ticket, Devolucion, FAQ)")

    # ========= RELACIONES =========

    # Producto - Ticket
    for _, row in tickets.iterrows():
        prod = graph.nodes.match("Producto", id=row["id_producto"]).first()
        ticket = graph.nodes.match("Ticket", id=row["id_ticket"]).first()
        if prod and ticket:
            graph.merge(Relationship(prod, "TIENE_TICKET", ticket))

    # Producto - Devolucion
    for _, row in devoluciones.iterrows():
        prod = graph.nodes.match("Producto", id=row["id_producto"]).first()
        dev = graph.nodes.match("Devolucion", id=row["id_devolucion"]).first()
        if prod and dev:
            graph.merge(Relationship(prod, "TIENE_DEVOLUCION", dev))

    # Producto - FAQ
    for item in faqs:
        prod = graph.nodes.match("Producto", id=item["id_producto"]).first()
        faq = graph.nodes.match("FAQ", id=item["id_faq"]).first()
        if prod and faq:
            graph.merge(Relationship(prod, "TIENE_FAQ", faq))

    print("🔗 Relaciones creadas (TIENE_TICKET, TIENE_DEVOLUCION, TIENE_FAQ)")
    print("🌐 Grafo construido en Neo4j")


In [ ]:
from py2neo import Node, Relationship

def limpiar_grafo():
    """Borra todos los nodos y relaciones de la base de Neo4j."""
    if graph is None:
        print("⚠️ No hay conexión a Neo4j")
        return
    graph.run("MATCH (n) DETACH DELETE n")
    print("🧹 Grafo limpiado")


def construir_grafo():
    """
    Construye un grafo sencillo con:
    Nodos: Producto, Ticket, Devolucion, FAQ
    Relaciones:
      (Producto)-[:TIENE_TICKET]->(Ticket)
      (Producto)-[:TIENE_DEVOLUCION]->(Devolucion)
      (Producto)-[:TIENE_FAQ]->(FAQ)
    Usa TODAS las filas de las tablas.
    """
    if graph is None:
        print("⚠️ No hay conexión a Neo4j")
        return

    limpiar_grafo()

    # ========= NODOS =========

    # Productos
    for _, row in productos.iterrows():
        nodo = Node(
            "Producto",
            id=row["id_producto"],
            nombre=row["nombre"],
            categoria=row["categoria"],
            subcategoria=row["subcategoria"],
            marca=row["marca"],
        )
        graph.merge(nodo, "Producto", "id")

    # Tickets
    for _, row in tickets.iterrows():
        nodo = Node(
            "Ticket",
            id=row["id_ticket"],
            tipo_problema=row["tipo_problema"],
            severidad=row["severidad"],
            categoria=row["categoria"],
            estado=row["estado"],
        )
        graph.merge(nodo, "Ticket", "id")

    # Devoluciones
    for _, row in devoluciones.iterrows():
        nodo = Node(
            "Devolucion",
            id=row["id_devolucion"],
            motivo=row["motivo"],
            estado=row["estado"],
            monto_reembolso=float(row["monto_reembolso"]),
        )
        graph.merge(nodo, "Devolucion", "id")

    # FAQs (viene de la lista 'faqs' que ya cargaste del JSON)
    for item in faqs:
        nodo = Node(
            "FAQ",
            id=item["id_faq"],
            categoria=item["categoria"],
            pregunta=item["pregunta"],
            respuesta=item["respuesta"],
        )
        graph.merge(nodo, "FAQ", "id")

    print("✅ Nodos creados (Producto, Ticket, Devolucion, FAQ)")

    # ========= RELACIONES =========

    # Producto - Ticket
    for _, row in tickets.iterrows():
        prod = graph.nodes.match("Producto", id=row["id_producto"]).first()
        ticket = graph.nodes.match("Ticket", id=row["id_ticket"]).first()
        if prod and ticket:
            graph.merge(Relationship(prod, "TIENE_TICKET", ticket))

    # Producto - Devolucion
    for _, row in devoluciones.iterrows():
        prod = graph.nodes.match("Producto", id=row["id_producto"]).first()
        dev = graph.nodes.match("Devolucion", id=row["id_devolucion"]).first()
        if prod and dev:
            graph.merge(Relationship(prod, "TIENE_DEVOLUCION", dev))

    # Producto - FAQ
    for item in faqs:
        prod = graph.nodes.match("Producto", id=item["id_producto"]).first()
        faq = graph.nodes.match("FAQ", id=item["id_faq"]).first()
        if prod and faq:
            graph.merge(Relationship(prod, "TIENE_FAQ", faq))

    print("🔗 Relaciones creadas (TIENE_TICKET, TIENE_DEVOLUCION, TIENE_FAQ)")
    print("🌐 Grafo construido en Neo4j")


In [ ]:
construir_grafo()


🧹 Grafo limpiado


In [ ]:
from py2neo import Graph
graph = Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))


In [ ]:
def crear_relaciones_batch():
    # 1) Producto - Ticket
    rows_tickets = tickets[["id_producto", "id_ticket"]].dropna().to_dict("records")
    query_tickets = """
    UNWIND $rows AS row
    MATCH (p:Producto {id: row.id_producto})
    MATCH (t:Ticket   {id: row.id_ticket})
    MERGE (p)-[:TIENE_TICKET]->(t)
    """
    graph.run(query_tickets, rows=rows_tickets)
    print(f"✔ Relaciones TIENE_TICKET creadas: {len(rows_tickets)} (aprox)")

    # 2) Producto - Devolucion
    rows_dev = devoluciones[["id_producto", "id_devolucion"]].dropna().to_dict("records")
    query_dev = """
    UNWIND $rows AS row
    MATCH (p:Producto   {id: row.id_producto})
    MATCH (d:Devolucion {id: row.id_devolucion})
    MERGE (p)-[:TIENE_DEVOLUCION]->(d)
    """
    graph.run(query_dev, rows=rows_dev)
    print(f"✔ Relaciones TIENE_DEVOLUCION creadas: {len(rows_dev)} (aprox)")

    # 3) Producto - FAQ
    rows_faq = [
        {"id_producto": f["id_producto"], "id_faq": f["id_faq"]}
        for f in faqs
    ]
    query_faq = """
    UNWIND $rows AS row
    MATCH (p:Producto {id: row.id_producto})
    MATCH (f:FAQ      {id: row.id_faq})
    MERGE (p)-[:TIENE_FAQ]->(f)
    """
    graph.run(query_faq, rows=rows_faq)
    print(f"✔ Relaciones TIENE_FAQ creadas: {len(rows_faq)} (aprox)")

    print("🎉 Relaciones creadas en batch.")


In [ ]:
crear_relaciones_batch()


NameError: name 'tickets' is not defined